In [1]:
from pathlib import Path
import textwrap

ROOT = Path(".").resolve()
(ROOT / "pages").mkdir(parents=True, exist_ok=True)
(ROOT / ".streamlit").mkdir(parents=True, exist_ok=True)

def w(p, s): p.write_text(textwrap.dedent(s).lstrip(), encoding="utf-8")

# Procfile
w(ROOT / "Procfile", """
web: streamlit run secure_app.py --server.port=$PORT --server.address=0.0.0.0 --server.headless=true
""")

# .streamlit/config.toml
w(ROOT / ".streamlit" / "config.toml", """
[server]
address = "0.0.0.0"
headless = true
enableXsrfProtection = true

[browser]
gatherUsageStats = false
""")

# requirements (Qiskit + Aer pinneados)
w(ROOT / "requirements.txt", """
streamlit==1.36.0
matplotlib==3.8.4
numpy==1.26.4
qiskit==1.1.1
qiskit-aer==0.14.2
""")

# .gitignore
w(ROOT / ".gitignore", """
__pycache__/
.env
*.pyc
.DS_Store
.streamlit/secrets.toml
""")

# README
w(ROOT / "README.md", """
# QC Qiskit Minimal (Streamlit + Railway)

## Variables de entorno (Railway → Variables)
- USER_USERNAME
- USER_PASSWORD_SHA256
- ADMIN_USERNAME
- ADMIN_PASSWORD_SHA256
- ADMIN_API_KEY  (opcional)

### SHA-256 (PowerShell)
$pwd = Read-Host "Contraseña"
$bytes = [Text.Encoding]::UTF8.GetBytes($pwd)
$hash  = [Security.Cryptography.SHA256]::Create().ComputeHash($bytes)
-join ($hash | ForEach-Object { $_.ToString("x2") })

### Despliegue
1) git init && git add . && git commit -m "init qiskit"
2) Sube a GitHub
3) Railway → New Project → Deploy from GitHub
4) Añade variables y Deploy
""")

# secure_app.py (login + bootstrap)
w(ROOT / "secure_app.py", r"""
import os, hashlib
import streamlit as st

st.set_page_config(page_title="QC Qiskit Minimal", layout="wide", page_icon="⚛️")

def _sha(s: str) -> str: return hashlib.sha256((s or "").encode("utf-8")).hexdigest()
def _get(name: str) -> str: return (os.getenv(name) or "").strip()

def _check(u_in: str, p_in: str, UENV: str, PENV: str) -> bool:
    return (u_in or "").strip() == _get(UENV) and _sha(p_in).lower() == _get(PENV).lower()

def login_box():
    with st.sidebar:
        st.header("🔒 Acceso")
        role = st.radio("Rol", ["user","admin"], horizontal=True)
        u = st.text_input("Usuario", value="")
        p = st.text_input("Contraseña", type="password", value="")
        api = None
        if role == "admin":
            st.caption("Admin puede exigir API key (opcional).")
            api = st.text_input("ADMIN_API_KEY", type="password", value="")
        go = st.button("Entrar", use_container_width=True, type="primary")

    if go:
        if role == "user":
            if _check(u,p,"USER_USERNAME","USER_PASSWORD_SHA256"):
                st.session_state.role = "user"
            else:
                st.sidebar.error("Credenciales de usuario no válidas.")
        else:
            if not _check(u,p,"ADMIN_USERNAME","ADMIN_PASSWORD_SHA256"):
                st.sidebar.error("Credenciales admin no válidas.")
            else:
                exp = _get("ADMIN_API_KEY")
                if exp and api != exp:
                    st.sidebar.error("La API key no coincide con ADMIN_API_KEY.")
                else:
                    st.session_state.role = "admin"
    return st.session_state.get("role")

def main():
    st.caption("🫀 servidor OK - build activo")
    role = login_box()
    if not role:
        st.info("Inicia sesión en el panel lateral.")
        return
    st.success(f"Sesión iniciada como **{role}**")
    import app_web2  # carga la app real

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        st.error("❌ La aplicación ha fallado al cargar.")
        st.exception(e)
""")

# app_web2.py (Qiskit puro, sin Plotly)
w(ROOT / "app_web2.py", r"""
import io
import numpy as np
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import streamlit as st

from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_bloch_multivector, plot_histogram, circuit_drawer
from qiskit_aer import Aer

st.title("⚛️ QC — Qiskit Minimal (Streamlit)")

@st.cache_resource(show_spinner=False)
def get_backends():
    return {
        "aer": Aer.get_backend("aer_simulator"),
        "qasm": Aer.get_backend("qasm_simulator"),
    }

@st.cache_data(show_spinner=False)
def fig_to_png(fig: Figure, dpi=300):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=dpi, bbox_inches="tight", pad_inches=0.05)
    buf.seek(0); return buf

def ket_from_angles(theta_deg: float, phi_deg: float):
    th = np.deg2rad(theta_deg); ph = np.deg2rad(phi_deg)
    a = np.cos(th/2.0); b = np.exp(1j*ph)*np.sin(th/2.0)
    v = np.array([a, b], dtype=complex)
    return v / np.linalg.norm(v)

tab1, tab2 = st.tabs(["1) Bloch 1-Qubit", "2) Circuitos + Simulación"])

# ---------- TAB 1: Bloch ----------
with tab1:
    st.subheader("Esfera de Bloch (Qiskit + Matplotlib)")
    c1, c2 = st.columns(2)
    with c1:
        th = st.slider("θ (deg)", 0.0, 180.0, 45.0, 0.1)
        ph = st.slider("φ (deg)", 0.0, 360.0, 30.0, 0.1)
    state = ket_from_angles(th, ph)
    sv = Statevector(state)

    try:
        fig = plot_bloch_multivector(sv)
        fig.set_size_inches(5.0, 5.0)
        st.pyplot(fig, use_container_width=False)
        st.download_button("⬇️ PNG (Bloch)", data=fig_to_png(fig), file_name="bloch.png", mime="image/png")
        plt.close(fig)
    except Exception as e:
        st.error("No se pudo renderizar la Bloch.")
        st.exception(e)

# ---------- TAB 2: Circuitos ----------
with tab2:
    st.subheader("Circuitos predeterminados (Aer)")
    left, right = st.columns([1,2], gap="large")
    with left:
        algo = st.selectbox("Algoritmo", ["Bell", "GHZ (n)", "QFT (n)", "Grover demo (n=3)"])
        if algo in ("GHZ (n)", "QFT (n)"):
            n = st.number_input("n (qubits)", min_value=2, value=3, step=1)
        shots = st.slider("Shots", 100, 4096, 1024, step=100)
        run = st.button("Generar + simular", type="primary", use_container_width=True)

    if run:
        try:
            # Construcción
            if algo == "Bell":
                qc = QuantumCircuit(2,2); qc.h(0); qc.cx(0,1); qc.measure([0,1],[0,1])
            elif algo == "GHZ (n)":
                qc = QuantumCircuit(n, n); qc.h(0)
                for i in range(n-1): qc.cx(i, i+1)
                qc.measure(range(n), range(n))
            elif algo == "QFT (n)":
                qc = QuantumCircuit(n, n)
                for j in range(n):
                    qc.h(j)
                    for k in range(j+1, n):
                        qc.cp(np.pi/2**(k-j), k, j)
                for i in range(n//2): qc.swap(i, n-1-i)
                qc.measure(range(n), range(n))
            else:  # Grover demo n=3 marcado "111"
                n = 3
                qc = QuantumCircuit(n, n); qc.h(range(n))
                oracle = QuantumCircuit(n)
                oracle.h(n-1); oracle.mcx(list(range(n-1)), n-1); oracle.h(n-1)
                diffuser = QuantumCircuit(n)
                diffuser.h(range(n)); diffuser.x(range(n))
                diffuser.h(n-1); diffuser.mcx(list(range(n-1)), n-1); diffuser.h(n-1)
                diffuser.x(range(n)); diffuser.h(range(n))
                qc.append(oracle.to_gate(label="Oracle"), range(n))
                qc.append(diffuser.to_gate(label="Diff"), range(n))
                qc.measure_all()

            # Diagrama
            try:
                figC = circuit_drawer(qc, output="mpl", style={'name':'mpl'})
                figC.set_size_inches(8.5, 2.8)
                st.pyplot(figC, use_container_width=True)
                st.download_button("⬇️ Diagrama (PNG)", data=fig_to_png(figC), file_name="circuit.png", mime="image/png")
                plt.close(figC)
            except Exception as e:
                st.warning("No se pudo dibujar el circuito.")
                st.exception(e)

            # Simulación
            b = get_backends()
            qasm = b["qasm"]
            res = qasm.run(transpile(qc, qasm), shots=int(shots)).result()
            counts = res.get_counts()

            figH = plot_histogram(counts)
            figH.set_size_inches(7.0, 3.0)
            st.pyplot(figH, use_container_width=True)
            st.download_button("⬇️ Histograma (PNG)", data=fig_to_png(figH), file_name="hist.png", mime="image/png")
            plt.close(figH)

        except Exception as e:
            st.error("Error al generar/simular.")
            st.exception(e)

st.caption("Proyecto minimal con Qiskit + Aer, sin Plotly. Listo para Railway.")
""")

# Página de diagnóstico
w(ROOT / "pages" / "00_diag.py", """
import os, sys, platform, streamlit as st
st.title("🔧 Diagnóstico")
st.write({
  "python": sys.version,
  "platform": platform.platform(),
  "railway_sha": os.getenv("RAILWAY_GIT_COMMIT_SHA","?"),
})
st.success("Si ves esto, el frontend JS funciona y el WebSocket está ok.")
""")

print("✅ Proyecto creado. Ahora:\n"
      "  git init && git add . && git commit -m \"init qiskit\"\n"
      "  (sube a GitHub y despliega en Railway; añade variables de entorno).")


✅ Proyecto creado. Ahora:
  git init && git add . && git commit -m "init qiskit"
  (sube a GitHub y despliega en Railway; añade variables de entorno).
